In [1]:
import findspark
findspark.init()
from pyspark import SparkContext
from pyspark.sql import *
from pyspark import SparkConf
import pyspark as ps
import pandas as pd

In [2]:
conf = (SparkConf()
            .setAppName("TestRdd") \
            .set('spark.driver.cores', '1') \
            .set('spark.executor.cores', '3') \
            .set('spark.driver.memory', '4G') \
            .set('spark.executor.memory', '16G'))\
            .set("spark.network.timeout", "15000s")
sc = SparkContext('local[*]', conf = conf)
sqlContext = SQLContext(sc)

In [3]:
data_reviews =  sqlContext.read.parquet('C:/Users/tomas/OneDrive/Escritorio/ORGA DE DATOS/review.parquet')
data_negocios = sqlContext.read.parquet('C:/Users/tomas/OneDrive/Escritorio/ORGA DE DATOS/business.parquet')

In [109]:
reviews = data_reviews.rdd
reviews_dict = reviews.map(lambda x: (x.business_id,[float(x.stars)]))

In [110]:
negocios = data_negocios.rdd
negocios_newOrleans = negocios.filter(lambda x: x.city == "New Orleans" and x.review_count > 10)\
                              .map(lambda x: (x.business_id,(x.review_count,x.categories)))\
                              .join(reviews_dict)

In [158]:
import numpy as np
score_promedio = negocios_newOrleans.reduceByKey(lambda x,y: ((x[0][0],x[0][1]),x[1]+y[1]))\ #Armo una lista con los rating
                   .map(lambda x: (x[0],(np.mean(x[1][1])-np.std(x[1][1]), x[1][0][1]))) #Calculo promedio y resto desviacion estandar


In [138]:
score_promedio.filter(lambda x: "French" in x[1][1])\
              .reduce(lambda x,y: x if x[1][0] > y[1][0] else y)
#CATEGORIA FRENCH

('7ZV31rGcDaT1pUKyvJF9Mg',
 (4.343714081985016,
  'Food, Restaurants, Bakeries, Breakfast & Brunch, French'))

In [139]:
score_promedio.filter(lambda x: "Bars" in x[1][1]).reduce(lambda x,y: x if x[1][0] > y[1][0] else y)
#CATEGORIA BARS

('2NImXwnAReGSwsr-qJV8EA', (5.0, 'Nightlife, Dive Bars, Bars'))

In [140]:
score_promedio.filter(lambda x: "Museums" in x[1][1]).reduce(lambda x,y: x if x[1][0] > y[1][0] else y)
#CATEGORIA MUSEUMS

('_Ac3-9QmcFDxNCHZRkOOlg', (5.0, 'Arts & Entertainment, Museums'))

In [ ]:
#MAPEO
#(business_id, (rating promedio, categorias))